In [1]:
from datetime import datetime as dt, timedelta
import numpy as np
import pandas as pd

# Ensure / Enforce Data Integrity

In [2]:
# gather initial datasets
broad = pd.read_csv('nominal_broad_dollar_index.csv')
bitcoin = pd.read_csv('bitcoin.csv')

In [3]:
bitcoin.shape, broad.shape

((2984, 6), (2657, 2))

In [4]:
bitcoin.day.min(), bitcoin.day.max(), broad.DATE.min(), broad.DATE.max()

('2016-01-01', '2024-03-01', '2014-01-02', '2024-03-08')

In [5]:
# check for duplicate dates
bitcoin.day.value_counts(), broad.DATE.value_counts()

(day
 2023-05-23    2
 2016-01-01    1
 2021-06-15    1
 2021-06-06    1
 2021-06-07    1
              ..
 2018-09-24    1
 2018-09-25    1
 2018-09-26    1
 2018-09-27    1
 2024-03-01    1
 Name: count, Length: 2983, dtype: int64,
 DATE
 2014-01-02    1
 2020-10-08    1
 2020-10-12    1
 2020-10-13    1
 2020-10-14    1
              ..
 2017-05-29    1
 2017-05-30    1
 2017-05-31    1
 2017-06-01    1
 2024-03-08    1
 Name: count, Length: 2657, dtype: int64)

In [6]:
# bitcoin has one duplicate
bitcoin[bitcoin.day == '2023-05-23']

,day,low,high,open,close,volume
2699,2023-05-23,26070.04,27225.0,27222.86,26321.4,17337.444316
2700,2023-05-23,26070.04,27225.0,27222.86,26321.4,17337.444316


In [7]:
# remove the duplicate date
bitcoin = bitcoin.drop(2700)
bitcoin.shape

(2983, 6)

In [8]:
# set datetimeindex for NBDI and bitcoin
broad['DATE'] = pd.to_datetime(broad['DATE'])
broad = broad.set_index('DATE')

bitcoin['day'] = pd.to_datetime(bitcoin['day'])
bitcoin = bitcoin.set_index('day')

In [9]:
# convert NBDI to float data type
broad['DTWEXBGS'] = broad['DTWEXBGS'].apply(lambda x: np.nan if x == '.' else x).astype('float')

In [10]:
# restrict to the same time range, 2016-01-01 to 2024-01-01
bitcoin_ranged = bitcoin['2016-01-01':'2024-01-01']
broad_ranged = broad['2016-01-01':'2024-01-01']

bitcoin_ranged.shape, broad_ranged.shape

((2923, 5), (2087, 1))

In [11]:
start = dt(2016, 1, 1)
end = dt(2024, 1, 2)
target_days = (end - start).days

In [12]:
# merge datasets
combined = bitcoin_ranged.merge(broad_ranged, left_index=True, right_index=True, how='outer', validate='one_to_one')
assert combined.shape[0] == target_days, f'Number of data points expected vs. actual: {target_days}, {combined.shape[0]}'

In [13]:
# rename nbdi field
combined = combined.rename(columns={'DTWEXBGS': 'nbdi'})

In [14]:
# all columns should be float type, check for nan values
for col in combined.columns:
    print(f'{col}:', combined[col].isna().sum())

low: 0
high: 0
open: 0
close: 0
volume: 0
nbdi: 932


In [15]:
# check summary stats in anticipation of imputing values
combined.nbdi.describe()

count    1991.000000
mean      115.719700
std         4.273181
min       106.487700
25%       112.716800
50%       115.173300
75%       118.588850
max       128.454400
Name: nbdi, dtype: float64

In [16]:
# impute values, forward fill due to historical nature of time, back fill any first missing values
combined = combined.ffill().bfill()

In [17]:
# stats should remain roughly equivalent after imputation
combined.nbdi.describe()

count    2923.000000
mean      115.716469
std         4.258462
min       106.487700
25%       112.713900
50%       115.186500
75%       118.427300
max       128.454400
Name: nbdi, dtype: float64

In [18]:
# ensure we have no further missing values
for col in combined.columns:
    print(f'{col}:', combined[col].isna().sum())

low: 0
high: 0
open: 0
close: 0
volume: 0
nbdi: 0


In [19]:
# save csv
combined.to_csv('bitcoin_data.csv')

In [20]:
combined.head()

,low,high,open,close,volume,nbdi
2016-01-01,432.41,437.56,435.67,435.40,3276.709621,114.1595
2016-01-02,425.02,435.75,435.40,431.91,3904.335318,114.1595
2016-01-03,431.37,435.79,431.90,433.85,5894.445723,114.1595
2016-01-04,430.00,435.64,433.84,433.34,5150.109476,114.1595
2016-01-05,428.15,433.46,433.32,430.87,5476.959959,114.2649


In [21]:
combined.shape

(2923, 6)

In [22]:
# we're only interested in close and volume values, drop the rest
dropped = combined.drop(columns=['low', 'high', 'open'])

In [23]:
dropped.head()

,close,volume,nbdi
2016-01-01,435.40,3276.709621,114.1595
2016-01-02,431.91,3904.335318,114.1595
2016-01-03,433.85,5894.445723,114.1595
2016-01-04,433.34,5150.109476,114.1595
2016-01-05,430.87,5476.959959,114.2649


In [24]:
dropped.to_csv('bitcoin_final.csv', index_label='date')

# Feature Extraction

In [25]:
bitcoin = pd.read_csv('bitcoin_final.csv', index_col='date')

In [26]:
bitcoin.shape

(2923, 3)

## Temporal Embedding

### Passage of Time

This should generically embed a feature that indicates the passage of time.

In [27]:
# Encode the passage of time
bitcoin.loc[:, 'days_elapsed'] = range(0, bitcoin.shape[0])

### Pre or Post Pandemic

As part of our hypothesis, we need to identify pre and post pandemic segments.  The delineation between the two is defined as the date when the World Health Organization officially declared a pandemic; March 11, 2020. We also want to encode the time passage of both epochs.

In [28]:
# use binary value to indicate if observation is post pandemic
bitcoin['post_pandemic'] = (bitcoin.index >= '2020-03-11')
bitcoin['post_pandemic'] = bitcoin.post_pandemic.astype(int)

In [29]:
# get days_elapsed property for day pandemic announced
center = bitcoin.loc['2020-03-11'].days_elapsed
bitcoin.loc[:, 'pandemic_elapsed'] = (bitcoin.days_elapsed - center)
bitcoin['pandemic_elapsed'] = bitcoin.pandemic_elapsed.astype(int)

### Calendar Features

Calendar features can help reveal relationships with seasonality and time-based trends

In [30]:
bitcoin[['calendar_year', 'calendar_month', 'calendar_day', 'calendar_weekday']] = 0

In [31]:
def calendar_components(r):
    d = dt.strptime(r.name, '%Y-%m-%d')
    r['calendar_day'] = d.day
    r['calendar_month'] = d.month
    r['calendar_year'] = d.year
    r['calendar_weekday'] = d.weekday()
    return r

bitcoin = bitcoin.copy()
bitcoin = bitcoin.apply(calendar_components, axis=1)

In [32]:
bitcoin.head()

,close,volume,nbdi,days_elapsed,post_pandemic,pandemic_elapsed,calendar_year,calendar_month,calendar_day,calendar_weekday
date,,,,,,,,,,
2016-01-01,435.40,3276.709621,114.1595,0.0,0.0,-1531.0,2016.0,1.0,1.0,4.0
2016-01-02,431.91,3904.335318,114.1595,1.0,0.0,-1530.0,2016.0,1.0,2.0,5.0
2016-01-03,433.85,5894.445723,114.1595,2.0,0.0,-1529.0,2016.0,1.0,3.0,6.0
2016-01-04,433.34,5150.109476,114.1595,3.0,0.0,-1528.0,2016.0,1.0,4.0,0.0
2016-01-05,430.87,5476.959959,114.2649,4.0,0.0,-1527.0,2016.0,1.0,5.0,1.0


## Price Features

### Close Price Lag

Data is non-stationary and resistant to attempts to make it stationary.  Price features should be chosen based on popular metrics used in financial trading.

In [33]:
immediate_lags = range(1, 8)
for i in immediate_lags:
    bitcoin[f'close_lag{i}'] = bitcoin.close.shift(i)

bitcoin.head()

,close,volume,nbdi,days_elapsed,post_pandemic,pandemic_elapsed,calendar_year,calendar_month,calendar_day,calendar_weekday,close_lag1,close_lag2,close_lag3,close_lag4,close_lag5,close_lag6,close_lag7
date,,,,,,,,,,,,,,,,,
2016-01-01,435.40,3276.709621,114.1595,0.0,0.0,-1531.0,2016.0,1.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-02,431.91,3904.335318,114.1595,1.0,0.0,-1530.0,2016.0,1.0,2.0,5.0,435.40,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-03,433.85,5894.445723,114.1595,2.0,0.0,-1529.0,2016.0,1.0,3.0,6.0,431.91,435.40,NaN,NaN,NaN,NaN,NaN
2016-01-04,433.34,5150.109476,114.1595,3.0,0.0,-1528.0,2016.0,1.0,4.0,0.0,433.85,431.91,435.40,NaN,NaN,NaN,NaN
2016-01-05,430.87,5476.959959,114.2649,4.0,0.0,-1527.0,2016.0,1.0,5.0,1.0,433.34,433.85,431.91,435.4,NaN,NaN,NaN


## Nominal Broad US Dollar Index Features (NBDI)

### NBDI Lags (30, 90, 180, 360)

Cross-correlation shows strong correlation between NBDI and Bitcoin close price with a 1+ year lag.  Multiple moving averages are suitable features in lieu of creating 365 (or more) features to capture the lag.

In [34]:
bitcoin['nbdi_ma_30'] = bitcoin.nbdi.rolling(30).mean()
bitcoin['nbdi_ma_90'] = bitcoin.nbdi.rolling(90).mean()
bitcoin['nbdi_ma_180'] = bitcoin.nbdi.rolling(180).mean()
bitcoin['nbdi_ma_365'] = bitcoin.nbdi.rolling(365).mean()

## Volume Features

### Volume lags (30, 90, 180, 360)

Similar to NBDI, trade volume has a strong correlation with a 1+ year lag and moving averages should be used.

In [35]:
bitcoin['volume_ma_30'] = bitcoin.volume.rolling(30).mean()
bitcoin['volume_ma_90'] = bitcoin.volume.rolling(90).mean()
bitcoin['volume_ma_180'] = bitcoin.volume.rolling(180).mean()
bitcoin['volume_ma_365'] = bitcoin.volume.rolling(365).mean()

In [36]:
bitcoin.to_csv('bitcoin_features.csv')